In [285]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
from numpy.random import randint
import numpy as np
from toolz import first

from itertools import groupby

import json

In [30]:
files = pd.Series(indir.glob('*'), name='files')

In [162]:
spec = {
    'early_morning': [ 1,2,3,4,5,6],
    'morning': [7,8,9,10,11,12],
    'afternoon': [13,14,15,16,],
    'evening': [17,18,19,20],
    'late_night' : [21,22,23, 0],
}
TIME_PERIODS = {hour: tp  for tp, hours in spec.items() for hour in hours}
TIME_PERIODS

{1: 'early_morning',
 2: 'early_morning',
 3: 'early_morning',
 4: 'early_morning',
 5: 'early_morning',
 6: 'early_morning',
 7: 'morning',
 8: 'morning',
 9: 'morning',
 10: 'morning',
 11: 'morning',
 12: 'morning',
 13: 'afternoon',
 14: 'afternoon',
 15: 'afternoon',
 16: 'afternoon',
 17: 'evening',
 18: 'evening',
 19: 'evening',
 20: 'evening',
 21: 'late_night',
 22: 'late_night',
 23: 'late_night',
 0: 'late_night'}

In [32]:
%%time 
indir = Path('/Users/wmcabee/Dropbox (Cognitive Scale)/NBC Analysis/data/NBC2/batches')

reader = indir.glob('*')
reader = map(pd.read_parquet, reader)
DATA = pd.concat(reader)

CPU times: user 21.1 s, sys: 7.8 s, total: 28.9 s
Wall time: 22 s


In [295]:
print('row_count:', len(DATA))

row_count: 6265392


In [222]:
def remove_bad_genres(df):
    genre = df.genre.value_counts(dropna=False)
    remove = {'None','', 'Not Set'}
    mask = df.genre.isin(remove)
    return  df[~mask].copy()

In [222]:
def mock_segments(df):
    M = df.groupby(['time_period','genre','segment']).size().unstack()
    df = M.sum(level=0)
    df = M.div(df, level=0)
    df = df.stack().to_frame('likelihood')

    df = df.round(2).reset_index().sort_index()
    df = df.set_index(['segment','time_period','genre']).likelihood
    reader = df.groupby(level=[0,1], )
    reader = ( (segment, time_period , v.droplevel([0,1]).to_dict()) for (segment, time_period), v in reader)
    reader = groupby(reader, key=lambda x: x[0] )
    return {g: { tp : periods for seg, tp, periods in v} for g, v in reader}

In [65]:
SEGMENTS = { segment_id : f'segment{segment_id:03d}' for segment_id in range(10)}
SEGMENTS

{0: 'segment000',
 1: 'segment001',
 2: 'segment002',
 3: 'segment003',
 4: 'segment004',
 5: 'segment005',
 6: 'segment006',
 7: 'segment007',
 8: 'segment008',
 9: 'segment009'}

In [279]:
np.random.seed(100)
df = DATA.sample(20000).drop_duplicates('mpid')
df['segment_id'] =  randint(0,10,len(df))
df['segment'] = df.segment_id.map(SEGMENTS)
df['event_start_dt'] = pd.to_datetime(df.event_start_unixtime_ms.astype(np.int).div(1000), unit="s")
df['video_end_hour'] = df['event_start_dt'].dt.hour
df['time_period'] = df.video_end_hour.map(TIME_PERIODS)
df = remove_bad_genres(df)
segment_lkup = mock_segments(df)
df['segment_detail'] = df.segment.map(segment_lkup)
SAMPLE = df

In [287]:
reader = SAMPLE.itertuples()
reader = ({'mpid': x.mpid, 'segment': {x.segment: x.segment_detail} }for x in reader)
MPIDS = list(reader)
example = MPIDS[1]

In [284]:
SAMPLE.ip

1158                                71.30.99.210
3336     2605:e000:1c03:21d2:1cd0:69f2:1392:51b4
40161    2605:a000:1203:632d:cc2b:1681:2c78:3d95
15066     2601:14a:4001:1d77:b8ff:4de9:adc7:7435
12406                              99.203.56.106
                          ...                   
17035                              24.186.72.123
41485                             209.201.32.200
38996                              108.39.38.131
24024                              75.185.250.96
14492       2601:644:500:de11:9d4:bc9f:75e8:f22d
Name: ip, Length: 19126, dtype: object

In [293]:
with open('response.json', 'w') as fh:
    json.dump(example, fh, indent=4)